In [34]:
from time import sleep
from itertools import chain, tee
import base64
from ipyleaflet import *
from IPython.display import display
from ipywidgets import FileUpload
import gpxpy

In [56]:
upload = FileUpload(accept='.gpx', multiple=True)
display(upload)

FileUpload(value={}, accept='.gpx', description='Upload', multiple=True)

In [57]:
def get_data(fname):
    points = []
    with open(fname, 'r') as f:
        gpx = gpxpy.parse(f)
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    points.append((point.latitude, point.longitude,
                                   point.speed, point.time))
    return points
tracks = [get_data(f) for f in upload.value]

In [58]:
class Extents:
    def __init__(self):
        t = tee(chain.from_iterable(tracks), 4)
        topleft = (min(p[0] for p in t[0]), min(p[1] for p in t[1]))
        botright = (max(p[0] for p in t[2]), max(p[1] for p in t[3]))
        width = botright[0] - topleft[0]
        height = botright[1] - topleft[1]
        self.center = (topleft[0] + 0.5 * width, topleft[1] + 0.5 * height)
        margin = (width * 0.1, height * 0.1)
        self.topleft = (topleft[0] - margin[0], topleft[1] - margin[1])
        self.botright = (botright[0] + margin[0], botright[1] + margin[1])
extents = Extents()

In [59]:
upload = FileUpload(accept='image/*', multiple=False)
display(upload)

FileUpload(value={}, accept='image/*', description='Upload')

In [66]:
class MapAnchors:
    def __init__(self):
        self.m = Map(center=extents.center, zoom=13)

        imname = next(iter(upload.value))
        data = 'data:image/png;base64,'
        with open(imname, 'rb') as f:
            data += base64.b64encode(f.read()).decode('utf-8')

        image = ImageOverlay(
            url=data,
            bounds=(extents.topleft, extents.botright),
            opacity=0.6
        )
        self.m.add_layer(image);

        self.image_markers = []
        for lat in (extents.topleft[0], extents.botright[0]):
            for long in (extents.topleft[1], extents.botright[1]):
                mark = Marker(location=(lat, long))
                self.m.add_layer(mark)
                self.image_markers.append(mark)
 
map_anchors = MapAnchors()
map_anchors.m

Map(center=[50.32000875, 30.57678325], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

In [71]:
class ImageAnchors:
    def __init__(self):
        self.m = Map(center=extents.center, zoom=13)

        track = Polyline(
            locations=[[p[0], p[1]] for p in tracks[0]],
            color='red',
            weight=2,
            fill=False,
            opacity=0.6
        )
        self.m.add_layer(track);

        self.image_markers = []
        for lat in (extents.topleft[0], extents.botright[0]):
            for long in (extents.topleft[1], extents.botright[1]):
                mark = Marker(location=(lat, long))
                self.m.add_layer(mark)
                self.image_markers.append(mark)
 
image_anchors = ImageAnchors()
image_anchors.m

Map(center=[50.32000875, 30.57678325], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

In [10]:
nb_steps = 10
step = 0.1
start_location = mark.location
end_location = (52.2, 10)

for i in range(nb_steps + 1):
    percentage_animation = i / nb_steps
    
    a = 1 - percentage_animation
    b = percentage_animation
    
    mark.location = (a * start_location[0] + b * end_location[0], a * start_location[1] + b * end_location[1])
    
    sleep(step)
